In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    # Оптимізована структура
    for i in range(hp.Int("num_layers", 2, 3)):  
        model.add(Dense(
            units=hp.Int(f"units_{i}", min_value=512, max_value=1024, step=256),
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(0.0001)
        ))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))  # Запобігання перенавчанню

    model.add(Dense(10, activation="softmax"))

    # Оптимізований learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Підбір гіперпараметрів
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=60,
    factor=3,
    directory="kt_fashion_mnist",
    project_name="mlp_tuning"
)

stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.5, min_lr=1e-5)

tuner.search(x_train, y_train, epochs=60, validation_data=(x_test, y_test), batch_size=2048, callbacks=[stop_early, reduce_lr])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(x_train, y_train, epochs=60, validation_data=(x_test, y_test), batch_size=2048, callbacks=[stop_early, reduce_lr])

test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Reloading Tuner from kt_fashion_mnist\mlp_tuning\tuner0.json
Epoch 1/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.5912 - loss: 1.3510 - val_accuracy: 0.6815 - val_loss: 1.1288 - learning_rate: 0.0010
Epoch 2/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8193 - loss: 0.5808 - val_accuracy: 0.7694 - val_loss: 0.9158 - learning_rate: 0.0010
Epoch 3/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.8431 - loss: 0.5102 - val_accuracy: 0.8045 - val_loss: 0.7755 - learning_rate: 0.0010
Epoch 4/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8529 - loss: 0.4747 - val_accuracy: 0.8363 - val_loss: 0.6630 - learning_rate: 0.0010
Epoch 5/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.8581 - loss: 0.4560 - val_accuracy: 0.8380 - val_loss: 0.5982 - learning_rate: 0.0010
Epoch 6/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.8670 - loss: 0.4297 - val_accuracy: 0.8412 - val_loss: 0.5647 - learning_rate: 0.0010
Epoch 7/60
30/30 ━━━━━━━━━━━━━━━━

Фінальна точність моделі 89,88%. Точність train_accuracy (94,29%)трохи вища за val_accuracy (89,83%).
val_loss залишається стабільним (≈ 0,35-0,36), що вказує на відсутність перенавчання.